In [11]:
from pathlib import Path

from gulpio2 import GulpDirectory
import numpy as np
import PIL.Image
from gulpio2.utils import img_to_jpeg_bytes

First we'll generate some random images and save them to disk to emulate having a real dataset.

In [12]:
n_images = 100
assert n_images % 10 == 0
images = np.random.randint(0, 255, (n_images, 224, 224, 3), dtype=np.uint8)
labels = np.repeat(np.arange(n_images // 10), n_images // 10)

In [13]:
for i, (img, label) in enumerate(zip(images, labels)):
    img_path = Path(f"fake-dataset/{label}/{i}.jpg")
    img_path.parent.mkdir(exist_ok=True, parents=True)
    with img_path.open("wb") as f:
        f.write(img_to_jpeg_bytes(img))

In [14]:
!tree fake-dataset

fake-dataset
├── 0
│   ├── 0.jpg
│   ├── 1.jpg
│   ├── 2.jpg
│   ├── 3.jpg
│   ├── 4.jpg
│   ├── 5.jpg
│   ├── 6.jpg
│   ├── 7.jpg
│   ├── 8.jpg
│   └── 9.jpg
├── 1
│   ├── 10.jpg
│   ├── 11.jpg
│   ├── 12.jpg
│   ├── 13.jpg
│   ├── 14.jpg
│   ├── 15.jpg
│   ├── 16.jpg
│   ├── 17.jpg
│   ├── 18.jpg
│   └── 19.jpg
├── 2
│   ├── 20.jpg
│   ├── 21.jpg
│   ├── 22.jpg
│   ├── 23.jpg
│   ├── 24.jpg
│   ├── 25.jpg
│   ├── 26.jpg
│   ├── 27.jpg
│   ├── 28.jpg
│   └── 29.jpg
├── 3
│   ├── 30.jpg
│   ├── 31.jpg
│   ├── 32.jpg
│   ├── 33.jpg
│   ├── 34.jpg
│   ├── 35.jpg
│   ├── 36.jpg
│   ├── 37.jpg
│   ├── 38.jpg
│   └── 39.jpg
├── 4
│   ├── 40.jpg
│   ├── 41.jpg
│   ├── 42.jpg
│   ├── 43.jpg
│   ├── 44.jpg
│   ├── 45.jpg
│   ├── 46.jpg
│   ├── 47.jpg
│   ├── 48.jpg
│   └── 49.jpg
├── 5
│   ├── 50.jpg
│   ├── 51.jpg
│   ├── 52.jpg
│   ├── 53.jpg
│   ├── 54.jpg
│   ├── 55.jpg
│   ├── 56.jpg
│   ├── 57.jpg
│   ├── 58.jpg
│   └── 59.jpg
├── 6
│   ├── 60.jpg
│   ├── 61.jpg
│   ├── 62.jpg
│   ├── 63

Now we'll gulp the dataset

In [15]:
%%bash
rm -rf fake-dataset-gulp
gulp2_image_folder \
    --images_per_chunk 100 \
    --num_workers 1 \
    --image_size 120 \
    --shuffle \
    fake-dataset/ \
    fake-dataset-gulp/

{'--help': False,
 '--image_size': '120',
 '--images_per_chunk': '100',
 '--num_workers': '1',
 '--shuffle': True,
 '--version': False,
 '<images_directory>': 'fake-dataset/',
 '<output_directory>': 'fake-dataset-gulp/'}


Chunks finished: 100%|██████████| 1/1 [00:00<00:00,  3.56chunk/s]


Now we have ingested our images into a gulp directory `fake-dataset-gulp`, let's see what's inside

In [16]:
!tree fake-dataset-gulp

fake-dataset-gulp
├── data_0.gulp
├── label2idx.json
└── meta_0.gmeta

0 directories, 3 files


Now we can load data and write a torch dataset class

In [17]:
gulp_dir = GulpDirectory('fake-dataset-gulp')

In [18]:
class GulpImageDataset:
    def __init__(self, gulp_dir: GulpDirectory, transform=None):
        self.gulp_dir = gulp_dir
        self.transform = transform if transform is not None else lambda x: x
        self.example_ids = list(gulp_dir.merged_meta_dict.keys())
    
    def __getitem__(self, idx):
        if isinstance(idx, int):
            example_id = self.example_ids[idx]
        else:
            example_id = idx
        imgs, meta = self.gulp_dir[example_id] 
        return self.transform(imgs[0]), meta
    
    def __len__(self):
        return len(self.gulp_dir.merged_meta_dict)

In [19]:
dataset = GulpImageDataset(gulp_dir)
print(len(dataset))

100


In [20]:
img, meta = dataset[0]
img.shape, meta

((120, 120, 3),
 {'id': '5-59.jpg', 'label': '5', 'path': 'fake-dataset/5', 'idx': 5})